In [1]:
import json, csv, re

In [3]:
f = open("Google_Result3.json")
google_data = json.load(f)

In [4]:
google_new_data = {}
for key in google_data.keys():
    values = google_data[key]
    new_v = []
    for v in values:
        v = re.split("http[s]*://[www.]*", v)[1]
        if v.endswith("/"):
            v = v[:-1]
        new_v.append(v)
    google_new_data[key]=new_v

google_data = google_new_data

In [5]:
f2 = open("hw1.json")
ask_data = json.load(f2)

In [6]:
ask_new_data = {}
for key in ask_data.keys():
    values = ask_data[key]
    new_v = []
    for v in values:
        v = re.split("http[s]*://[www.]*", v)[1]
        if v.endswith("/"):
            v = v[:-1]
        new_v.append(v)
    ask_new_data[key]=new_v

ask_data = ask_new_data

In [7]:
number = 0
outputs = []

total_overlap = 0
total_overlap_per = 0
total_rho = 0

for key in ask_data.keys():
    number += 1
    if key not in google_data.keys():
        print(key + " not found")
        break
    else:
        ask_res=ask_data[key]
        google_res=google_data[key]
        overlap = list(set(ask_res) & set(google_res))
        # number of overlapping results
        n_overlap = len(overlap)
        total_overlap += n_overlap
        
        # Percent overlap
        per_overlap = n_overlap / 10.0 * 100
        total_overlap_per += per_overlap
        
        # Spearman rho
        if n_overlap == 0:
            rho = 0
        elif n_overlap == 1:
            for q in overlap:
                ask_rank = ask_res.index(q)
                google_rank = google_res.index(q)
                if ask_rank == google_rank:
                    rho = 1.0
                else:
                    rho = 0.0
        else:
            sum_d_sqr = 0
            for q in overlap:
                ask_rank = ask_res.index(q)
                google_rank = google_res.index(q)
                d_sqr = (google_rank - ask_rank) ** 2
                sum_d_sqr += d_sqr
            rho = 1 - (6*sum_d_sqr)/(n_overlap*(n_overlap**2 - 1))
        total_rho += rho
        
        output = ["Query "+str(number), " "+str(n_overlap), " "+str(per_overlap), " "+str(rho)]
        outputs.append(output)

In [9]:
header = ['Queries', 'Number of Overlapping Results', 'Percent Overlap', "Spearman Coefficient"]

avg_overlap = " " + str(total_overlap/number)
avg_overlap_per = " " + str(total_overlap_per/number)
avg_rho = " " + str(total_rho/number)

summary = ['Averages', avg_overlap, avg_overlap_per, avg_rho]

with open("hw1.csv", "w", newline='') as csvf:
    writer = csv.writer(csvf, delimiter=",")
    writer.writerow(header)
    for out in outputs:
        writer.writerow(out)
    writer.writerow(summary)